In [49]:
import os
# os.environ["MKL_DEBUG_CPU_TYPE"] = "5"
# os.environ["MKL_SERVICE_FORCE_INTEL"] = "1"
import sys
import glob
import numpy as np
import pandas as pd

sys.path.append('/Users/villasenor/science/github/jvillasr/MINATO')
from minato import ravel, myRC

import importlib
importlib.reload(ravel)

import cProfile

In [51]:
import matplotlib
print(matplotlib.get_backend())

module://matplotlib_inline.backend_inline


In [50]:
results_file = 'RVs_results.csv'

# Load the results from the file if it exists
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file, sep='\t')
else:
    # Initialize an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['Star_field_ID', 'nepochs', 'dRV', 'sigma_d', 'Bin', 'Period', 'LS_Pow', 'P_qflag'])
print('results_df: ', results_df)


path='sb1_data/'
spectrum_files = sorted(glob.glob(f"{path}*Combined.fits"))

print('spectrum_files: ', spectrum_files)

cProfile.run("ravel.SLfit(spectrum_files, path='', lines=[4026, 4102, 4131, 4144, 4340, 4388, 4471, 4481, 4542, 4553], plots=True, shift=3, neblines=[])", 'profiling_results.out')
# print('star_path from SLfit: ', star_path)


results_df:  Empty DataFrame
Columns: [Star_field_ID, nepochs, dRV, sigma_d, Bin, Period, LS_Pow, P_qflag]
Index: []
spectrum_files:  ['sb1_data/BLOeM_1-100_01_Combined.fits', 'sb1_data/BLOeM_1-100_02_Combined.fits', 'sb1_data/BLOeM_1-100_03_Combined.fits', 'sb1_data/BLOeM_1-100_04_Combined.fits', 'sb1_data/BLOeM_1-100_05_Combined.fits', 'sb1_data/BLOeM_1-100_06_Combined.fits', 'sb1_data/BLOeM_1-100_07_Combined.fits', 'sb1_data/BLOeM_1-100_08_Combined.fits', 'sb1_data/BLOeM_1-100_09_Combined.fits']
*** SB2 set to:  False  ***

these are the lines:  [4026, 4102, 4131, 4144, 4340, 4388, 4471, 4481, 4542, 4553]
Fitting line  4026
Fitting line  4102
Fitting line  4131
Fitting line  4144
Fitting line  4340
Fitting line  4388
Fitting line  4471
Fitting line  4481
Fitting line  4542
Fitting line  4553
 ...fit done
 ...plots done



In [17]:
import pstats

p = pstats.Stats('profiling_results.out')
p.sort_stats('cumulative').print_stats(50)  # This will print the top 10 functions by cumulative time

Sat Jul  6 16:47:49 2024    profiling_results.out

         24549887 function calls (24116108 primitive calls) in 13.469 seconds

   Ordered by: cumulative time
   List reduced from 2326 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   13.482   13.482 {built-in method builtins.exec}
        1    0.002    0.002   13.482   13.482 <string>:1(<module>)
        1    0.004    0.004   13.480   13.480 /Users/villasenor/science/github/jvillasr/MINATO/minato/ravel.py:299(SLfit)
       10    0.000    0.000   10.801    1.080 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/pyplot.py:1129(savefig)
       10    0.000    0.000    8.751    0.875 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/figure.py:3234(savefig)
       10    0.001    0.000    8.751    0.875 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/backend_bases.py:2052(print_figure)
       30    0.000  

In [16]:
p.sort_stats('tottime').print_stats(50)

Sat Jul  6 16:47:49 2024    profiling_results.out

         24549887 function calls (24116108 primitive calls) in 13.469 seconds

   Ordered by: internal time
   List reduced from 2326 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       66    1.066    0.016    1.066    0.016 {method 'encode' of 'ImagingEncoder' objects}
    13832    0.536    0.000    0.664    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/dviread.py:621(<listcomp>)
   227665    0.385    0.000    0.385    0.000 {method 'reduce' of 'numpy.ufunc' objects}
   301018    0.378    0.000    0.409    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/__init__.py:690(_get)
    12960    0.245    0.000    1.621    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/ticker.py:2077(_raw_ticks)
   928429    0.212    0.000    0.412    0.000 /opt/miniconda3/envs/py310/lib/python3.10/_weakrefset.py:63(__iter__)


In [27]:
# def is_my_module(function):
#     return function.__module__ == __name__
# def is_my_module(filename):
#     return filename.startswith(os.path.dirname(os.path.abspath(__file__)))
def is_my_module(filename):
    return filename.startswith(os.getcwd())

class MyProfiler(cProfile.Profile):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._my_module_functions = set()

    def enable(self):
        for function in self.getstats():
            if is_my_module(function):
                self._my_module_functions.add(function)
        super().enable()

    def disable(self):
        super().disable()
        # for function in self.getstats():
        #     if function not in self._my_module_functions:
        #         function.__call__ = function.__wrapped__

In [ ]:
profiler = MyProfiler()
profiler.run('ravel.SLfit(spectrum_files, path="", lines=[4026, 4102, 4131, 4144, 4340, 4388, 4471, 4481, 4542, 4553], shift=3, neblines=[])')
profiler.dump_stats('my_profiling_results.out')

In [23]:
p = pstats.Stats('my_profiling_results.out')
p.sort_stats('tottime').print_stats(50)  # This will print the top 10 functions by cumulative time

Sat Jul  6 17:58:12 2024    my_profiling_results.out

         24549834 function calls (24116043 primitive calls) in 13.226 seconds

   Ordered by: internal time
   List reduced from 2328 to 50 due to restriction <50>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       66    1.059    0.016    1.059    0.016 {method 'encode' of 'ImagingEncoder' objects}
    13832    0.528    0.000    0.659    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/dviread.py:621(<listcomp>)
   227665    0.383    0.000    0.383    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    41752    0.299    0.000    0.327    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/cbook.py:285(process)
    12960    0.241    0.000    1.596    0.000 /opt/miniconda3/envs/py310/lib/python3.10/site-packages/matplotlib/ticker.py:2077(_raw_ticks)
   928429    0.211    0.000    0.408    0.000 /opt/miniconda3/envs/py310/lib/python3.10/_weakrefset.py:63(__iter_

In [29]:
# Print filenames and check result
for stat in p.stats.items():
    filename = stat[0][0]
    print(f"Filename: {filename}, Check result: {is_my_module(filename)}")

Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: ~, Check result: False
Filename: 

In [28]:
# Filter out entries that don't belong to your module
filtered_stats = [stat for stat in p.stats.items() if is_my_module(stat[0][0])]

# Sort by cumulative time and print the top 10
filtered_stats.sort(key=lambda stat: stat[1][3], reverse=True)
for func, (cc, nc, tt, ct, callers) in filtered_stats[:10]:
    print(f"{func[2]}({func[1]}): {ct}")